<a href="https://colab.research.google.com/github/Rezvan1344/ACIT-4070/blob/main/h%C3%A5rek%20code%20mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
import pickle
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

#!pip install statsmodels
import statsmodels.stats.api as sms

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mnist_train.csv to mnist_train.csv


In [ ]:
import pandas as pd
import io
df1 = pd.read_csv(io.BytesIO(uploaded['mnist_train.csv']))

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mnist_test.csv to mnist_test.csv


In [ ]:
import pandas as pd
import io
df2 = pd.read_csv(io.BytesIO(uploaded['mnist_test.csv']))

In [ ]:
dataset = pd.concat([df1,df2])

In [ ]:
dataset.values[:] = StandardScaler().fit_transform(dataset)

In [ ]:
dataset=shuffle(dataset)
col = 'label' 
target = dataset[col]
datasetX = dataset.drop(col, axis=1)

In [ ]:
import numpy as np
clients=3
dataArr = np.array_split(datasetX, clients)
targetArr = np.array_split(target, clients)

In [ ]:
ml = []
fl = []
av = []
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [ ]:
for runs in range (0,20):
    x_train, x_test, y_train, y_test = train_test_split(datasetX, target, test_size = 0.2)
    model = LogisticRegression(max_iter=1000)
    model.fit(x_train, y_train)
    score=model.score(x_test,y_test)
    ml.append(score)
    averCoef = model.coef_[0]
    #print(averCoef)
    averCoef = averCoef*0 # Initializing array
    #print(averCoef)

In [ ]:
aver=0
for i in range (0,clients):
  x_trainA, x_testA, y_trainA, y_testA = train_test_split(dataArr[i], targetArr[i], test_size = 0.2)

  model = LogisticRegression(max_iter=500)
  model.fit(x_trainA, y_trainA)
  score=model.score(x_testA,y_testA)
  # print(i,score)
  aver = aver + score
  coef = model.coef_[0]
  averCoef = averCoef + coef
    
averCoef = averCoef/clients
#print(averCoef)
aver = aver/clients
av.append(aver)
#print("Average score", clients,"clients: ",aver)

model.coef_[0] = averCoef
#print(model.coef_[0])
score=model.score(x_test,y_test)
#print("FL:",score)
fl.append(score)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


ML: 0.8801577380952381 0.0030293803553369694 (0.8784905552533516, 0.8818249209371246)
AV: 0.8694021855581744 0.0005356760231413071 (0.8625957763357432, 0.8762085947806055)
FL: 0.8766785714285714 0.007178571428571423 (0.7854661731420411, 0.9678909697151017)


In [ ]:
print("ML:",np.mean(ml),np.std(ml),sms.DescrStatsW(ml).tconfint_mean())
print("AV:",np.mean(av),np.std(av),sms.DescrStatsW(av).tconfint_mean())
print("FL:",np.mean(fl),np.std(fl),sms.DescrStatsW(fl).tconfint_mean())

ML: 0.8802301587301589 0.002745630303714658 (0.8770737844786571, 0.8833865329816606)
AV: 0.8688665095350331 0.0 (nan, nan)
FL: 0.8695 0.0 (nan, nan)


/usr/local/lib/python3.7/dist-packages/statsmodels/stats/weightstats.py:228: RuntimeWarning: invalid value encountered in double_scalars
  return std / np.sqrt(self.sum_weights - 1)
